<a href="https://colab.research.google.com/github/ddodds42/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/DAVID_DODDS_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import numpy as np
import scipy.stats as sp
from scipy.stats import ttest_1samp, ttest_ind, ttest_ind_from_stats, ttest_rel


In [40]:
# Load and clean the data

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data')
print(df.shape)
df.head()

(434, 17)


,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [0]:
col_names = ['party', 'handicapped_infants', 'water_project', 'budget',
             'physician_fee_freeze', 'el_salvador_aid',
             'religious_groups_in_schools', 'anti_satellite_test_ban',
             'aid_nicaraguan_contras', 'mx_missile', 'immigration',
             'synfuels-corp-cutback', 'ed_spending', 'superfund_right_to_sue',
             'crime', 'duty_free_exports', 'south_africa']

In [42]:
df.columns = col_names
print(df.shape)
df.head()

(434, 17)


,party,handicapped_infants,water_project,budget,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_nicaraguan_contras,mx_missile,immigration,synfuels-corp-cutback,ed_spending,superfund_right_to_sue,crime,duty_free_exports,south_africa
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [43]:
df = df.replace({'?':np.NaN, 'n':0, 'y':1})
df.head()

,party,handicapped_infants,water_project,budget,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_nicaraguan_contras,mx_missile,immigration,synfuels-corp-cutback,ed_spending,superfund_right_to_sue,crime,duty_free_exports,south_africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
1,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
4,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0


In [44]:
dem = df[df['party'] == 'democrat']
rep = df[df['party'] == 'republican']

dem.head()

,party,handicapped_infants,water_project,budget,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_nicaraguan_contras,mx_missile,immigration,synfuels-corp-cutback,ed_spending,superfund_right_to_sue,crime,duty_free_exports,south_africa
1,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
4,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [45]:
rep.head()

,party,handicapped_infants,water_project,budget,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_nicaraguan_contras,mx_missile,immigration,synfuels-corp-cutback,ed_spending,superfund_right_to_sue,crime,duty_free_exports,south_africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
6,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
9,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0
10,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,NaN,NaN


In [46]:
rep.describe()

,handicapped_infants,water_project,budget,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_nicaraguan_contras,mx_missile,immigration,synfuels-corp-cutback,ed_spending,superfund_right_to_sue,crime,duty_free_exports,south_africa
count,164.000000,147.000000,163.000000,164.000000,164.000000,165.000000,161.000000,156.000000,164.000000,164.000000,159.000000,154.000000,157.000000,160.000000,155.000000,145.000000
mean,0.189024,0.503401,0.134969,0.987805,0.951220,0.896970,0.242236,0.153846,0.115854,0.554878,0.132075,0.870130,0.859873,0.981250,0.090323,0.655172
std,0.392727,0.501698,0.342744,0.110092,0.216069,0.304924,0.429773,0.361963,0.321030,0.498501,0.339643,0.337257,0.348230,0.136067,0.287573,0.476960
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [47]:
df.describe()

,handicapped_infants,water_project,budget,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_nicaraguan_contras,mx_missile,immigration,synfuels-corp-cutback,ed_spending,superfund_right_to_sue,crime,duty_free_exports,south_africa
count,422.000000,386.000000,423.000000,423.000000,419.000000,423.000000,420.000000,419.000000,412.000000,427.000000,414.000000,403.000000,409.000000,417.000000,406.000000,330.000000
mean,0.443128,0.502591,0.598109,0.416076,0.503580,0.640662,0.569048,0.577566,0.502427,0.503513,0.362319,0.421836,0.508557,0.592326,0.428571,0.812121
std,0.497345,0.500642,0.490861,0.493490,0.500585,0.480375,0.495800,0.494537,0.500602,0.500574,0.481252,0.494466,0.500539,0.491992,0.495482,0.391209
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [48]:
dem.describe()

,handicapped_infants,water_project,budget,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_nicaraguan_contras,mx_missile,immigration,synfuels-corp-cutback,ed_spending,superfund_right_to_sue,crime,duty_free_exports,south_africa
count,258.000000,239.000000,260.000000,259.000000,255.000000,258.000000,259.000000,263.000000,248.000000,263.000000,255.000000,249.000000,252.000000,257.000000,251.000000,185.000000
mean,0.604651,0.502092,0.888462,0.054054,0.215686,0.476744,0.772201,0.828897,0.758065,0.471483,0.505882,0.144578,0.289683,0.350195,0.637450,0.935135
std,0.489876,0.501045,0.315405,0.226562,0.412106,0.500430,0.420224,0.377317,0.429121,0.500138,0.500949,0.352383,0.454518,0.477962,0.481697,0.246956
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [49]:
# 3. Using hypothesis testing, find an issue that republicans support more than
# democrats with p < 0.01

print(df['ed_spending'].mean())
print(rep['ed_spending'].mean())
print(dem['ed_spending'].mean())
ttest_ind(rep['ed_spending'], dem['ed_spending'], nan_policy='omit')

# It would seem that this proposal for education spending was more favored by
# Republicans. It does not mean that they are more in favor of education spending,
# it just means that they liked this particular spending package better than
# democrats.

# t_stat = 20.414
# pvalue = 4.9676 x 10^-64

0.4218362282878412
0.8701298701298701
0.14457831325301204


Ttest_indResult(statistic=20.414298768685285, pvalue=4.967619782338976e-64)

In [56]:
# 2. Using hypothesis testing, find an issue that democrats support more than
# republicans with p < 0.01

print(df['south_africa'].mean())
print(dem['south_africa'].mean())
print(rep['south_africa'].mean())
ttest_ind(rep['south_africa'], dem['south_africa'], nan_policy='omit')

# The bill on exports to South Africa was favored by a majority of both parties,
# but Democrats favored it more strongly than Republicans.

# t_stat = -6.89
# pvalue = 2.806 x 10^-11

0.8121212121212121
0.9351351351351351
0.6551724137931034


Ttest_indResult(statistic=-6.893205322931943, pvalue=2.8060959129817107e-11)

In [57]:
# 4. Using hypothesis testing, find an issue where the difference between
# republicans and democrats has p > 0.1 (i.e. there may not be much of a
# difference)

print(df['water_project'].mean())
print(dem['water_project'].mean())
print(rep['water_project'].mean())
ttest_ind(dem['water_project'], rep['water_project'], nan_policy='omit')

# the p value here is like 9 times the threshold of 0.1. Both parties were split
# almost exactly 50/50 on this issue.

0.5025906735751295
0.502092050209205
0.5034013605442177


Ttest_indResult(statistic=-0.02491808700047811, pvalue=0.9801332440121653)

In [0]:
from statistics import mean

sb = [0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
 1, 0, 1, 0, 1, 1, 0, 0, 0]

m = mean(sb)

In [51]:
sp.ttest_1samp(sb, 0.5)

Ttest_1sampResult(statistic=2.364321853156195, pvalue=0.02207003200903075)